#Import all libraries

In [1]:
!pip --q install datasets
!pip --q install deep_translator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 853.0 kB/s eta 0:00:00


In [2]:
import warnings
warnings.filterwarnings("ignore")
import json
import os
import copy

from deep_translator import GoogleTranslator
from datasets import DatasetDict,Dataset

import pandas as pd
import torch

# Download and explore the original dataset

 [visualqa](https://visualqa.org/)

In [3]:
versionType  ="v2"          #VQA v2.0 dataset
taskType    ='OpenEnded'    #type of annotations in the JSON file (OpenEnded).
dataType    ='mscoco'       #source of the images (mscoco or abstract_v002).
dataSubType = "train2014"   #type of data subtype (e.g. train2014/val2014/test2015 for mscoco, train2015/val2015 for abstract_v002).


annotationsName = f"{versionType}_Annotations_Train_{dataType}"




## Download

In [4]:
!mkdir original-dataset

In [5]:
%cd /content/original-dataset/

/content/original-dataset


In [6]:
annotationsLink = f"https://s3.amazonaws.com/cvmlp/vqa/{dataType}/vqa/{annotationsName}.zip"
!wget {annotationsLink}
!unzip -q {annotationsName+".zip"}
!rm {annotationsName+".zip"}

--2024-04-27 14:07:54--  https://s3.amazonaws.com/cvmlp/vqa/mscoco/vqa/v2_Annotations_Train_mscoco.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.93.229, 52.216.113.93, 52.217.228.8, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.93.229|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21708861 (21M) [application/zip]
Saving to: ‘v2_Annotations_Train_mscoco.zip’

v2_Annotations_Trai 100%[===================>]  20.70M  53.4MB/s    in 0.4s    

2024-04-27 14:07:55 (53.4 MB/s) - ‘v2_Annotations_Train_mscoco.zip’ saved [21708861/21708861]





---

## Explore



In [7]:
annFile     = f"{versionType}_{dataType}_{dataSubType}_annotations.json"

In [8]:
with open(annFile, "r") as f:
    ann_dict = json.load(f)

ann_dict.keys()

dict_keys(['info', 'license', 'data_subtype', 'annotations', 'data_type'])

In [9]:
annotations_ds = Dataset.from_list(ann_dict['annotations'])
annotations_ds

Dataset({
    features: ['question_type', 'multiple_choice_answer', 'answers', 'image_id', 'answer_type', 'question_id'],
    num_rows: 443757
})

In [10]:
annotations_ds[0]

{'question_type': 'what is this',
 'multiple_choice_answer': 'net',
 'answers': [{'answer': 'net', 'answer_confidence': 'maybe', 'answer_id': 1},
  {'answer': 'net', 'answer_confidence': 'yes', 'answer_id': 2},
  {'answer': 'net', 'answer_confidence': 'yes', 'answer_id': 3},
  {'answer': 'netting', 'answer_confidence': 'yes', 'answer_id': 4},
  {'answer': 'net', 'answer_confidence': 'yes', 'answer_id': 5},
  {'answer': 'net', 'answer_confidence': 'yes', 'answer_id': 6},
  {'answer': 'mesh', 'answer_confidence': 'maybe', 'answer_id': 7},
  {'answer': 'net', 'answer_confidence': 'yes', 'answer_id': 8},
  {'answer': 'net', 'answer_confidence': 'yes', 'answer_id': 9},
  {'answer': 'net', 'answer_confidence': 'yes', 'answer_id': 10}],
 'image_id': 458752,
 'answer_type': 'other',
 'question_id': 458752000}

In [11]:
len(annotations_ds)

443757



---

# Translation

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
%cd /content/
!mkdir /content/drive/MyDrive/annotations-translated-dataset

/content
mkdir: cannot create directory ‘/content/drive/MyDrive/annotations-translated-dataset’: File exists


In [14]:
%cd /content/drive/MyDrive/annotations-translated-dataset/

/content/drive/MyDrive/annotations-translated-dataset




---
**Annotations Translation**


In [15]:
torch.set_num_threads(1)

In [16]:
def annotations_translation(batch):
  try:
    sample = batch['multiple_choice_answer']
    batch["multiple_choice_answer"] =  GoogleTranslator(source='auto', target='ar').translate_batch(sample)
    print("len batch:",len(sample))
  except:
     batch["multiple_choice_answer"] = sample
     print("An except occurred")
     annotations_translation(batch)


  return batch

In [17]:
lenann = len(annotations_ds)
lenann

443757

In [18]:
# range(50000,100000)
# range(200000,250000)
# range(350000,400000)

In [19]:
annotations_translation_ds = annotations_ds.select(range(350000,400000)).map(
            annotations_translation,
            batched=True,
            num_proc = 2,
            desc="Annotations (multiple_choice_answer) Translation",
        )



Annotations (multiple_choice_answer) Translation (num_proc=2):   0%|          | 0/50000 [00:00<?, ? examples/s…

len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000
len batch: 1000


In [20]:
annotations_translation_ds[-1]

{'question_type': 'are these',
 'multiple_choice_answer': 'نعم',
 'answers': [{'answer': 'yes', 'answer_confidence': 'yes', 'answer_id': 1},
  {'answer': 'yes', 'answer_confidence': 'maybe', 'answer_id': 2},
  {'answer': 'no', 'answer_confidence': 'maybe', 'answer_id': 3},
  {'answer': 'no', 'answer_confidence': 'yes', 'answer_id': 4},
  {'answer': 'no', 'answer_confidence': 'yes', 'answer_id': 5},
  {'answer': 'yes', 'answer_confidence': 'maybe', 'answer_id': 6},
  {'answer': 'yes', 'answer_confidence': 'yes', 'answer_id': 7},
  {'answer': 'no', 'answer_confidence': 'yes', 'answer_id': 8},
  {'answer': 'yes', 'answer_confidence': 'maybe', 'answer_id': 9},
  {'answer': 'yes', 'answer_confidence': 'yes', 'answer_id': 10}],
 'image_id': 117514,
 'answer_type': 'yes/no',
 'question_id': 117514000}

---
**Save Annotations**

In [21]:
ann_dict['annotations'] =  annotations_translation_ds.to_list()

In [22]:
# section = 2
# section = 5
# section = 8

In [23]:
section = 8

In [24]:
with open(f'translated_{section}_{annFile}', 'w') as f:
  json.dump(ann_dict, f)


In [25]:
json.load(open(f'translated_{section}_{annFile}','r'))

{'info': {'description': 'This is v2.0 of the VQA dataset.',
  'url': 'http://visualqa.org',
  'version': '2.0',
  'year': 2017,
  'contributor': 'VQA Team',
  'date_created': '2017-04-26 17:07:13'},
 'license': {'url': 'http://creativecommons.org/licenses/by/4.0/',
  'name': 'Creative Commons Attribution 4.0 International License'},
 'data_subtype': 'train2014',
 'annotations': [{'question_type': 'what',
   'multiple_choice_answer': 'كيك',
   'answers': [{'answer': 'cake', 'answer_confidence': 'yes', 'answer_id': 1},
    {'answer': 'cake', 'answer_confidence': 'yes', 'answer_id': 2},
    {'answer': 'cake', 'answer_confidence': 'yes', 'answer_id': 3},
    {'answer': 'dessert', 'answer_confidence': 'maybe', 'answer_id': 4},
    {'answer': 'dessert', 'answer_confidence': 'yes', 'answer_id': 5},
    {'answer': 'cake', 'answer_confidence': 'yes', 'answer_id': 6},
    {'answer': 'cake', 'answer_confidence': 'yes', 'answer_id': 7},
    {'answer': 'cake', 'answer_confidence': 'yes', 'answer_i